## Generate Climos

This process runs [generate_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_climos-generate-climatological-means), which creates files with climatological means/standard deviations of input data from a netcdf file.

In [1]:
from birdy import WPSClient
import requests
import os
from urllib.request import urlopen, urlretrieve
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from bs4 import BeautifulSoup
import re
from wps_tools.testing import get_target_url

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("thunderbird")
print(f"Using thunderbird on {url}")

Using thunderbird on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps


In [3]:
thunderbird = WPSClient(url)

In [3]:
# NBVAL_IGNORE_OUTPUT
# Check info on `generate_climos` process
thunderbird.generate_climos?

Signature:
thunderbird.generate_climos(
    netcdf,
    operation,
    dry_run=None,
    convert_longitudes=True,
    split_vars=True,
    split_intervals=True,
    loglevel='INFO',
    climo=None,
    resolutions=None,
)
Docstring:
Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
operation : {'mean', 'std'}string
    Operation to perform on the datasets
climo : {'6190', '7100', '8110', '2020', '2050', '2080'}string
    Year ranges
resolutions : {'all', 'yearly', 'seasonal', 'monthly'}string
    Temporal Resolutions
convert_longitudes : boolean
    Transform longitude range from [0, 360) to [-180, 180)
split_vars : boolean
    Generate a separate file for each dependent variable in the file
split_intervals : boolean
    Gener

## Single File run
**Dry Run -** Checks file to ensure compatible with process

In [4]:
# Set up variables for thunderbird.generate_climos
seasonal_opendap = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc'
annual_opendap = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc'
operation = 'mean'
climo = '6190'
resolutions = 'yearly'
dry_run = True

# Dry run process
dry_output = thunderbird.generate_climos(
    netcdf=seasonal_opendap, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

In [5]:
# Process dry run output
req = requests.get(dry_output.get()[0])
metalink = BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl
print(metalink)

<metaurl mediatype="text/plain">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/20bbedfa-1589-11eb-b305-0242ac120008/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100_dry.txt
    </metaurl>


In [6]:
# Test dry run output is as expected
url = metalink.get_text()
with urlopen(url) as f:
    output_data = f.read().decode('utf-8')

expected_items = ['6190', 'CMIP5', 'PCIC', 'CanESM2', 'historical', 'rcp85', 'r1i1p1', 'fdd', 'seasonal']
for item in expected_items:
    assert item in output_data

**Normal Run**

In [7]:
# generate climos
output = thunderbird.generate_climos(
    netcdf=seasonal_opendap, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

In [8]:
# Process normal output
req = requests.get(output.get()[0])
metalink = BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl
print(metalink)

<metaurl mediatype="application/x-netcdf">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/2436e192-1589-11eb-b305-0242ac120008/fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc
    </metaurl>


In [9]:
""" Helper function to test netCDF file output -
Creating a 30 year average with this process given the parameters should squash the time 
dimension down from x (where x is the number of days in the input data) to 1 in the output data. 
"""
def test_nc_data(url):
    with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
        urlretrieve(url, tmp_file.name)
        output_data = Dataset(tmp_file.name)

        assert output_data.dimensions['time'].size == 1

In [10]:
# Test normal output data
url = metalink.get_text()
test_nc_data(url)

## Multiple File Run
**Dry Run -** Checks files to ensure compatible with process

In [11]:
# process dry run for multiple files
dry_output = thunderbird.generate_climos(
    netcdf=[seasonal_opendap, annual_opendap], 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

In [12]:
req = requests.get(dry_output.get()[0])
metalinks = BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).find_all('metaurl')
print(metalinks)

[<metaurl mediatype="text/plain">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/250ca9f8-1589-11eb-b305-0242ac120008/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100_dry.txt
    </metaurl>, <metaurl mediatype="text/plain">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/250cd766-1589-11eb-b305-0242ac120008/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100_dry.txt
    </metaurl>]


In [13]:
# Test dry output for multiple files
assert len(metalinks) == 2
    
for link, tr in zip(metalinks, ['seasonal', 'yearly']):
    url = link.get_text()
    with urlopen(url) as f:
        output_data = f.read().decode('utf-8')
    
    assert re.search(r'time_resolution: {}'.format(tr), output_data)

**Normal Run**

In [14]:
# Process normal output for multiple files

output = thunderbird.generate_climos(
    netcdf=[seasonal_opendap,annual_opendap], 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

In [15]:
req = requests.get(output.get()[0])
metalinks = BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).find_all('metaurl')
print(metalinks)

[<metaurl mediatype="application/x-netcdf">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/2a602c54-1589-11eb-b305-0242ac120008/fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc
    </metaurl>, <metaurl mediatype="application/x-netcdf">
     https://docker-dev03.pcic.uvic.ca/wpsoutputs/2a606cbe-1589-11eb-b305-0242ac120008/gdd_aClimMean_CanESM2_rcp85_r1i1p1_19610101-19901231.nc
    </metaurl>]


In [16]:
# Test multiple files
assert len(metalinks) == 2

for link in metalinks:
    url = link.get_text()
    test_nc_data(url)